In [11]:
import random

from pyjobshop import Model

m = Model()
jobs = [m.add_job() for _ in range(4)]
machines = [m.add_machine() for _ in range(2)]

for job in jobs:
    operations = [m.add_operation(job=job) for _ in range(4)]

    for op in operations:
        for machine in machines:
            duration = random.randint(1, 10)
            m.add_processing_time(machine, op, duration)

result = m.solve()